# Decision Trees

**Outline:**
- Implement impurity measure
- Implement a Decision Tree from Scratch
  - Find Best Split
  - How to recursively build the tree?
- Steps on how to build the Decision Tree

In [1]:
# Let's start by importing the necessary libraries
import numpy as np
import pandas as pd
from collections import Counter

## Impurity Measures

The following are three impurity indices:

$$
\begin{aligned}
\text{Entropy} &= -\sum_{i=0}^{c-1}  p_i(t) \log_2 p_i(t) \\
\text{Gini index} &= 1 - \sum_{i=0}^{c-1}  p_i(t)^2 \\
\text{Classification error} &= 1 - \max_i p_i(t)
\end{aligned}
$$

where $p_i(t)$ is the **relative frequency** of training instances of class $i$ at a node $t$ and $c$ is the number of classes.

By convention, we set $0 \log_2 0 = 0$ in entropy calculations.


All three impurity indices equal 0 when all the records at a node belong to the same class.

All three impurity indices reach their maximum value when the classes are evenly distributed among the child nodes.

In [2]:
# Define functions for entropy, Gini Index, and Classification Error
def entropy(y):
    """
    Calculate the entropy of a set of labels.
    """
    counts = np.bincount(y)
    probabilities = counts / len(y)
    entropy_value = -np.sum([p * np.log2(p) for p in probabilities if p > 0])
    return entropy_value

def gini_index(y):
    """
    Calculate the Gini Index of a set of labels.
    """
    counts = np.bincount(y)
    probabilities = counts / len(y)
    gini = 1 - np.sum([p ** 2 for p in probabilities])
    return gini

def classification_error(y):
    """
    Calculate the Classification Error of a set of labels.
    """
    counts = np.bincount(y)
    probabilities = counts / len(y)
    error = 1 - np.max(probabilities)
    return error


In [3]:
# Test the functions
y = np.array([0, 0, 1, 1, 1])
print("Entropy:", entropy(y))
print("Gini Index:", gini_index(y))
print("Classification Error:", classification_error(y))

Entropy: 0.9709505944546686
Gini Index: 0.48
Classification Error: 0.4


## How to find the best split?
Let's take the case of Entropy.

To find the best split, we calculate the weighted average entropy after a split

The code calculates the **weighted average entropy** after a split. Here's how it works:
- `left_y` and `right_y` represent the labels (or target variable) of the left and right subsets of the data after the split.
- `len(left_y)` and `len(right_y)` give the number of samples in each subset.
- `n` is the total number of samples before the split (i.e., `n = len(left_y) + len(right_y)`).

The split criterion (entropy) is calculated as a **weighted sum** of the entropies of the two subsets:

$$ \text{Weighted Entropy} = \frac{\text{len(left_y)}}{n} \times \text{entropy(left_y)} + \frac{\text{len(right_y)}}{n} \times \text{entropy(right_y)} $$

- The terms `len(left_y) / n` and `len(right_y) / n` are the proportions of samples in each subset.
- The entropy of each subset is calculated separately, and then the weighted sum is taken. The weighted sum reflects how "impure" the split is.

In [6]:
def _split_criterion(self, left_y, right_y):
    """
    Calculate the criterion for splitting.
    """
    n = len(left_y) + len(right_y)
    if self.criterion == 'entropy':
        return (len(left_y) / n) * entropy(left_y) + (len(right_y) / n) * entropy(right_y)
    elif self.criterion == 'gini':
        return (len(left_y) / n) * gini_index(left_y) + (len(right_y) / n) * gini_index(right_y)
    elif self.criterion == 'classification_error':
        return (len(left_y) / n) * classification_error(left_y) + (len(right_y) / n) * classification_error(right_y)

## Implement DecisionTree

## Step 1: Class Initialization

First, we'll create a `DecisionTree` class and initialize the necessary parameters for our tree such as the splitting criterion, maximum depth, and the tree structure.

```python
class DecisionTree:
    def __init__(self, criterion='entropy', max_depth=None):
        self.criterion = criterion
        self.max_depth = max_depth
        self.tree = None
```

## Step 2: Fitting the Model

Next, we implement the `fit()` method, which is used to train the decision tree on the given dataset `X` (features) and `y` (target labels). The `fit()` method will call a helper function `_build_tree()` to recursively build the tree.

```python
def fit(self, X, y):
    self.tree = self._build_tree(X, y, depth=0)
```

## Step 3: Recursive Tree Building

Now, we implement the logic to build the decision tree recursively. The `_build_tree()` method checks for the base cases (such as when all labels are the same or the maximum depth is reached) and finds the best feature and threshold for splitting the data.

```python
def _build_tree(self, X, y, depth):
    num_samples, num_features = X.shape
    if num_samples == 0 or len(set(y)) == 1 or (self.max_depth is not None and depth >= self.max_depth):
        leaf_value = self._most_common_label(y)
        return leaf_value

    best_feature, best_threshold = self._best_split(X, y)

    left_indices = X[:, best_feature] < best_threshold
    right_indices = X[:, best_feature] >= best_threshold

    left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
    right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

    return (best_feature, best_threshold, left_subtree, right_subtree)
```

## Step 4: Finding the Best Split

In the next step, we implement `_best_split()` to find the feature and threshold that provide the best split of the data. We'll loop over all features and thresholds to calculate the metric (entropy, gini, etc.) that defines the best split.

```python
def _best_split(self, X, y):
    num_samples, num_features = X.shape
    best_metric = float('inf')
    best_feature = None
    best_threshold = None

    for feature in range(num_features):
        thresholds = np.unique(X[:, feature])
        for threshold in thresholds:
            left_indices = X[:, feature] < threshold
            right_indices = X[:, feature] >= threshold
            if len(left_indices) == 0 or len(right_indices) == 0:
                continue

            metric = self._split_criterion(y[left_indices], y[right_indices])
            if metric < best_metric:
                best_metric = metric
                best_feature = feature
                best_threshold = threshold

    return best_feature, best_threshold
```

## Step 5: Calculating the Split Criterion

Here, we define the `_split_criterion()` method to compute the quality of the split using the chosen criterion (entropy, gini, or classification error). Depending on the selected criterion, the appropriate calculation will be performed.

```python
def _split_criterion(self, left_y, right_y):
    n = len(left_y) + len(right_y)
    if self.criterion == 'entropy':
        return (len(left_y) / n) * entropy(left_y) + (len(right_y) / n) * entropy(right_y)
    elif self.criterion == 'gini':
        return (len(left_y) / n) * gini_index(left_y) + (len(right_y) / n) * gini_index(right_y)
    elif self.criterion == 'classification_error':
        return (len(left_y) / n) * classification_error(left_y) + (len(right_y) / n) * classification_error(right_y)
```

## Step 6: Most Common Label in a Node

To handle cases where we need to create a leaf node, we implement `_most_common_label()`. This method returns the most frequent label from the target values.

```python
from collections import Counter

def _most_common_label(self, y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common
```

## Step 7: Making Predictions

Now, we implement the `predict()` method, which allows us to make predictions on new data points. This method will call `_predict()` to traverse the decision tree for each input.

```python
def predict(self, X):
    return [self._predict(inputs) for inputs in X]

def _predict(self, inputs):
    node = self.tree
    while isinstance(node, tuple):
        feature, threshold, left_subtree, right_subtree = node
        if inputs[feature] < threshold:
            node = left_subtree
        else:
            node = right_subtree
    return node
```

In [7]:
# Implement the Decision Tree from scratch
class DecisionTree:
    def __init__(self, criterion='entropy', max_depth=None):
        self.criterion = criterion
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        if num_samples == 0 or len(set(y)) == 1 or (self.max_depth is not None and depth >= self.max_depth):
            leaf_value = self._most_common_label(y)
            return leaf_value

        # Find the best split
        best_feature, best_threshold = self._best_split(X, y)

        # Split the data
        left_indices = X[:, best_feature] < best_threshold
        right_indices = X[:, best_feature] >= best_threshold

        # Recursively build the left and right subtrees
        left_subtree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_subtree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return (best_feature, best_threshold, left_subtree, right_subtree)

    def _best_split(self, X, y):
        num_samples, num_features = X.shape
        best_metric = float('inf')
        best_feature = None
        best_threshold = None

        for feature in range(num_features):
            thresholds = np.unique(X[:, feature])
            for threshold in thresholds:
                left_indices = X[:, feature] < threshold
                right_indices = X[:, feature] >= threshold
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue
                
                metric = self._split_criterion(y[left_indices], y[right_indices])
                if metric < best_metric:
                    best_metric = metric
                    best_feature = feature
                    best_threshold = threshold
        
        return best_feature, best_threshold

    def _split_criterion(self, left_y, right_y):
        """
        Calculate the criterion for splitting.
        """
        n = len(left_y) + len(right_y)
        if self.criterion == 'entropy':
            return (len(left_y) / n) * entropy(left_y) + (len(right_y) / n) * entropy(right_y)
        elif self.criterion == 'gini':
            return (len(left_y) / n) * gini_index(left_y) + (len(right_y) / n) * gini_index(right_y)
        elif self.criterion == 'classification_error':
            return (len(left_y) / n) * classification_error(left_y) + (len(right_y) / n) * classification_error(right_y)

    def _most_common_label(self, y):
        """
        Get the most common label in the dataset.
        """
        counter = Counter(y)
        most_common = counter.most_common(1)[0][0]
        return most_common

    def predict(self, X):
        return [self._predict(inputs) for inputs in X]

    def _predict(self, inputs):
        node = self.tree
        while isinstance(node, tuple):
            feature, threshold, left_subtree, right_subtree = node
            if inputs[feature] < threshold:
                node = left_subtree
            else:
                node = right_subtree
        return node


## Test on a 'sample' dataset

In [5]:
# Sample dataset
X = np.array([[2, 3],
                [1, 1],
                [4, 5],
                [3, 2],
                [3, 4]])
y = np.array([0, 0, 1, 1, 1])

# Initialize and fit the decision tree
clf = DecisionTree(criterion='entropy', max_depth=3)
clf.fit(X, y)

# Make predictions
predictions = clf.predict(X)
print("Predictions:", predictions)

Predictions: [0, 0, 1, 1, 1]
